In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

2025-10-26 14:20:44.312316: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-10-26 14:20:44.312545: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-26 14:20:44.347515: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-26 14:20:45.107463: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off,

# Building Complex Models Using the Functional API
One example of a nonsequential neural network is a Wide & Deep neural network

<img src="https://raw.githubusercontent.com/sambitmukherjee/handson-ml3-pytorch/main/chapter10/Figure_10-13.png" width="480">

In [2]:
housing = fetch_california_housing()
 
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, random_state=42)

In [3]:
normalization_layer = tf.keras.layers.Normalization()
hidden_layer1 = tf.keras.layers.Dense(30, activation="relu")
hidden_layer2 = tf.keras.layers.Dense(30, activation="relu")
concat_layer = tf.keras.layers.Concatenate()
output_layer = tf.keras.layers.Dense(1)

In [4]:
input_ = tf.keras.layers.Input(shape=X_train.shape[1:])
normalized = normalization_layer(input_)
hidden1 = hidden_layer1(normalized)
hidden2 = hidden_layer2(hidden1)
concat = concat_layer([normalized, hidden2])
output = output_layer(concat)

2025-10-26 14:20:45.865908: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [5]:
model = tf.keras.Model(inputs=[input_], outputs=[output])

But what if you want to send a subset of the features through the wide path and a different subset (possibly overlapping) through the deep path, In this case, one solution is to use multiple inputs. For example, suppose we want to send five features through the wide path (features 0 to 4), and six features through the deep path (features 2 to 7). We can do this as follows:


<img src="https://raw.githubusercontent.com/sambitmukherjee/handson-ml3-pytorch/main/chapter10/Figure_10-14.png" width="480">

In [6]:
input_wide = tf.keras.layers.Input(shape=[5]) # features 0 to 4
input_deep = tf.keras.layers.Input(shape=[6]) # features 2 to 7

norm_layer_wide = tf.keras.layers.Normalization()
norm_layer_deep = tf.keras.layers.Normalization()

norm_wide = norm_layer_wide(input_wide)
norm_deep = norm_layer_deep(input_deep)

hidden1 = tf.keras.layers.Dense(30, activation="relu")(norm_deep)
hidden2 = tf.keras.layers.Dense(30, activation="relu")(hidden1)

concat = tf.keras.layers.concatenate([norm_wide, hidden2])
output = tf.keras.layers.Dense(1)(concat)

model = tf.keras.Model(inputs=[input_wide, input_deep], outputs=[output])

There are a few things to note in this example, compared to the previous one:

Each Dense layer is created and called on the same line. This is a common practice, as it makes the code more concise without losing clarity. However, we can’t do this with the Normalization layer since we need a reference to the layer to be able to call its adapt() method before fitting the model.



We used tf.keras.layers.concatenate(), which creates a Concatenate layer and calls it with the given inputs.


We specified inputs=[input_wide, input_deep] when creating the model, since there are two inputs

In [7]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
model.compile(loss="mse", optimizer=optimizer, metrics=["RootMeanSquaredError"])

In [8]:
X_train_wide, X_train_deep = X_train[:, :5], X_train[:, 2:]
X_valid_wide, X_valid_deep = X_valid[:, :5], X_valid[:, 2:]

X_test_wide, X_test_deep = X_test[:, :5], X_test[:, 2:]
X_new_wide, X_new_deep = X_test_wide[:3], X_test_deep[:3]

In [9]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit((X_train_wide, X_train_deep), y_train, epochs=20, validation_data=((X_valid_wide, X_valid_deep), y_valid))

mse_test = model.evaluate((X_test_wide, X_test_deep), y_test)
y_pred = model.predict((X_new_wide, X_new_deep))

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - RootMeanSquaredError: 1.1646 - loss: 1.3563 - val_RootMeanSquaredError: 0.9548 - val_loss: 0.9117
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.7311 - loss: 0.5346 - val_RootMeanSquaredError: 1.1819 - val_loss: 1.3970
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.6724 - loss: 0.4521 - val_RootMeanSquaredError: 0.9344 - val_loss: 0.8730
Epoch 4/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 910us/step - RootMeanSquaredError: 0.6363 - loss: 0.4048 - val_RootMeanSquaredError: 1.1261 - val_loss: 1.2681
Epoch 5/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.6249 - loss: 0.3906 - val_RootMeanSquaredError: 1.2635 - val_loss: 1.5963
Epoch 6/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.6167 - loss: 0.3803 - val_RootMeanSquaredError: 1.5900 - val_loss: 2.5281
Epoch 7/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - RootMeanSquaredError: 0.6072 - l

There are also many use cases in which you may want to have multiple outputs:

The task may demand it. For instance, you may want to locate and classify the main object in a picture. This is both a regression tasks and a classification task.

Similarly, you may have multiple independent tasks based on the same data. Sure, you could train one neural network per task, but in many cases you will get better results on all tasks by training a single neural network with one output per task. This is because the neural network can learn features in the data that are useful across tasks. For example, you could perform multitask classification on pictures of faces, using one output to classify the person’s facial expression (smiling, surprised, etc.) and another output to identify whether they are wearing glasses or not.


Another use case is as a regularization technique (i.e., a trainingconstraint whose objective is to reduce overfitting and thus improve the model’s ability to generalize). For example, you may want to add an auxiliary output in a neural network architecture (see Figure 10-15) to ensure that the underlying part of the network learns something useful on its own, without relying on the rest of the network.

<img src="https://raw.githubusercontent.com/sambitmukherjee/handson-ml3-pytorch/main/chapter10/Figure_10-15.png" width="480">

Adding an extra output is quite easy: we just connect it to the appropriate layer and add it to the model’s list of outputs. For example, the following code builds the network represented in Figure 10-15:

In [10]:
output = tf.keras.layers.Dense(1)(concat)
aux_output = tf.keras.layers.Dense(1)(hidden2)

model = tf.keras.Model(inputs=[input_wide, input_deep],outputs=[output, aux_output])

In [17]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)

model.compile(loss=("mse", "mse"), loss_weights=(0.9, 0.1), optimizer=optimizer, metrics=[['RootMeanSquaredError'], ['RootMeanSquaredError']])

In [18]:
norm_layer_wide.adapt(X_train_wide)
norm_layer_deep.adapt(X_train_deep)

history = model.fit(
    (X_train_wide, X_train_deep), (y_train, y_train), epochs=20, 
    validation_data=((X_valid_wide, X_valid_deep), (y_valid, y_valid))
)

Epoch 1/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - dense_6_RootMeanSquaredError: 1.2875 - dense_6_loss: 1.6569 - dense_7_RootMeanSquaredError: 1.1649 - dense_7_loss: 1.3565 - loss: 1.6275 - val_dense_6_RootMeanSquaredError: 0.7273 - val_dense_6_loss: 0.5288 - val_dense_7_RootMeanSquaredError: 0.7847 - val_dense_7_loss: 0.6156 - val_loss: 0.5376
Epoch 2/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_6_RootMeanSquaredError: 0.6836 - dense_6_loss: 0.4672 - dense_7_RootMeanSquaredError: 0.7465 - dense_7_loss: 0.5573 - loss: 0.4763 - val_dense_6_RootMeanSquaredError: 0.8174 - val_dense_6_loss: 0.6680 - val_dense_7_RootMeanSquaredError: 0.8215 - val_dense_7_loss: 0.6746 - val_loss: 0.6689
Epoch 3/20
363/363 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_6_RootMeanSquaredError: 0.6376 - dense_6_loss: 0.4066 - dense_7_RootMeanSquaredError: 0.7059 - dense_7_loss: 0.4984 - loss: 0.4157 - val_dense_6_RootMeanSquaredError: 0.6552 - val_dense_6_loss: 0.4292 - val_dense_7_RootMeanSquaredError: 0.8

In [19]:
eval_results = model.evaluate((X_test_wide, X_test_deep), (y_test, y_test))
weighted_sum_of_losses, main_loss, aux_loss, main_rmse, aux_rmse = eval_results

162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_6_RootMeanSquaredError: 0.5646 - dense_6_loss: 0.3184 - dense_7_RootMeanSquaredError: 0.6106 - dense_7_loss: 0.3729 - loss: 0.3242 


If you set return_dict=True, then evaluate() will return a dictionary instead of a big tuple.

In [20]:
y_pred_main, y_pred_aux = model.predict((X_new_wide, X_new_deep))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


In [21]:
y_pred_tuple = model.predict((X_new_wide, X_new_deep))
y_pred = dict(zip(model.output_names, y_pred_tuple))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step


# Using the Subclassing API to Build Dynamic Models